# We are going to cover some of the most fundamental concepts of tensors using TensorFlow
## More specifically, we are going to cover:
* Introduction to tensors
* Getting information from Tensors
* Manipulating Tensors
* Tensors & NumPy
* USing @tf.function (a way to speed up regular python functions)
* Using GPUs with Tensorflow(or TPU)
* Exercises to try yourself.

# Tensorflow Docker instance command
nvidia-docker run -it -p 8888:8888 tensorflow_customized:latest-gpu-jupyter


In [3]:
!nvidia-smi

Mon Jul 11 02:43:07 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.73.05    Driver Version: 510.73.05    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| N/A   47C    P8    N/A /  N/A |      0MiB /  2048MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
import tensorflow as tf
# Check if GPU available
tf.test.is_gpu_available(
    cuda_only=False, min_cuda_compute_capability=None
)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

# Introduction to Tensors

In [5]:
# Import Tensorflow
import tensorflow as tf
print(tf.__version__)

2.9.1


In [6]:
# Creating tensors with tf.constant()
scalar = tf.constant(7)
scalar

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [7]:
# Check the number of dimensions of a tensor (ndim stands for number of dimensions)
scalar.ndim

0

In [8]:
# Create a vector
vector = tf.constant([10,10])
vector

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 10], dtype=int32)>

In [9]:
# Check the dimensions of our vector
vector.ndim

1

In [10]:
# Create a matrix (has more than one dimensions)
matrix = tf.constant([[10,7],
                      [7,10]])
matrix

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 7, 10]], dtype=int32)>

In [11]:
matrix.ndim

2

In [12]:
# Create another matrix
another_matrix = tf.constant([[10., 7.],
                              [3., 2.],
                              [8., 9.]], dtype=tf.float16) # specify the data type with tensor
another_matrix

<tf.Tensor: shape=(3, 2), dtype=float16, numpy=
array([[10.,  7.],
       [ 3.,  2.],
       [ 8.,  9.]], dtype=float16)>

In [13]:
# What's the number of dimensions of another matrix?
another_matrix.ndim

2

# The total number of dimensions is the number of elements in shape.

In [14]:
# Let's create a tensor
tensor = tf.constant([[[1,2,3],
                       [4,5,6]],
                      [[7,8,9],
                       [10,11,12]],
                      [[13,14,15],
                       [16,17,18]]])
tensor

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ 1,  2,  3],
        [ 4,  5,  6]],

       [[ 7,  8,  9],
        [10, 11, 12]],

       [[13, 14, 15],
        [16, 17, 18]]], dtype=int32)>

In [15]:
tensor.ndim

3

What we've created so far:
* Scalar: a single number
* Vector: a number with direction (e.g. wind speed and direction)
* Matrix: a 2-dimensional array of numbers
* Tensor: an n-dimensional array of numbers (where n can be any number, a 0-dimensional tensor is a scalar, a 1-dimensional tensor is a vector)

### Creating tensors with `tf.variable`

In [16]:
# Create the same tensor with tf.Variable() as above
changeable_tensor = tf.Variable([10,7])
unchangeable_tensor = tf.constant([10,7])
changeable_tensor, unchangeable_tensor

(<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  7], dtype=int32)>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([10,  7], dtype=int32)>)

In [17]:
#Let's try change one of elements in our changeable tensor
changeable_tensor[0] = 7
changeable_tensor

TypeError: 'ResourceVariable' object does not support item assignment

In [18]:
# How about we try .assign()
changeable_tensor[0].assign(7)
changeable_tensor

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([7, 7], dtype=int32)>

In [19]:
# Let's try to change unchangeable tensor
unchangeable_tensor[0].assign(7)
unchangeable_tensor

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'assign'

🔑 **Note:** Rarely in practice will you need to decide wheer to use `tf.constant` or `tf.variable` to create tensors, as tensorflow dows this for you, if in doubt, use `tf.constant` and change it later if needed.

## Creating random tensors

Random tensors are tensors of some arbitrary size which contain random numbers.
Use `tf.random` function

In [23]:
# Create two random (but the same) tensors
random_1 = tf.random.Generator.from_seed(7) #set seed for reproducibility
random_1 = random_1.normal(shape=(3,2))
random_2 = tf.random.Generator.from_seed(7)
random_2 = random_2.normal(shape=(3,2))

#Are they equal?
random_1, random_2, random_1==random_2

(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-1.3240396 ,  0.2878567 ],
        [-0.8757901 , -0.08857017],
        [ 0.69211644,  0.84215707]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-1.3240396 ,  0.2878567 ],
        [-0.8757901 , -0.08857017],
        [ 0.69211644,  0.84215707]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=bool, numpy=
 array([[ True,  True],
        [ True,  True],
        [ True,  True]])>)

## Shuffle the order of elements in a tensor
Use `random.shuffle`, shuffles along first direction


In [40]:
# Shuffle a tensor ( valuable for when you want to shuffle the data so that the inherent order does not affect learnings)
not_shuffled = tf.constant([[10,7],
                            [3,4],
                            [2,5]])
# Shuffled our not shuffled tensor
tf.random.shuffle(not_shuffled) #shuffled along axis 0 i.e, 3

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 2,  5],
       [ 3,  4]], dtype=int32)>

In [44]:
# Shuffled our not shuffled tensor
tf.random.set_seed(42) #global level seed
tf.random.shuffle(not_shuffled, seed=42) #operation level seed

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

## Exercise 🔨 🔨

Read through Tensorflow documentation on random seed generation:
https://www.tensorflow.org/api_docs/python/tf/random/set_seed

Practice writing 5 random tensors and shuffle them.


In [42]:
not_shuffled

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>